In [1]:
# =========================
# 1) IMPORT THƯ VIỆN (theo 2.3.2 Bài làm mẫu)
# =========================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# =========================
# 2) NẠP DỮ LIỆU & KHẢO SÁT (giống bước 1–2 của bài mẫu)
# =========================
CSV_PATH = "/kaggle/input/customer-behaviour/Customer_Behaviour.csv"  # <-- đổi đường dẫn nếu cần
df = pd.read_csv(CSV_PATH)

print(df.head())
print("\nThiếu dữ liệu theo cột:\n", df.isna().sum())

# Chuẩn hoá tên cột cho chắc (loại khoảng trắng/viết hoa)
df.columns = [c.strip().replace(" ", "").replace("-", "").replace("/", "") for c in df.columns]
# Sau chuẩn hóa, kỳ vọng: ['UserID','Gender','Age','EstimatedSalary','Purchased']

# Loại bỏ ID (không phải feature dự báo)
drop_id_cols = [c for c in df.columns if c.lower() in ["userid","id","user_id","userID".lower()]]
df = df.drop(columns=drop_id_cols, errors="ignore")

# Mục tiêu & đặc trưng
target_col = "Purchased"
feature_cols = ["Gender", "Age", "EstimatedSalary"]

# Một số dataset ghi 'Purchased' là string; ép về int 0/1
if df[target_col].dtype == "O":
    df[target_col] = df[target_col].astype(str).str.strip().str.lower().map({
        "1":1, "yes":1, "y":1, "true":1, "purchase":1, "purchased":1,
        "0":0, "no":0, "n":0, "false":0, "not purchased":0
    }).fillna(df[target_col]).astype(int)

X = df[feature_cols].copy()
y = df[target_col].astype(int).copy()

# =========================
# 3) TÁCH TẬP TRAIN/TEST (80/20) (giống bài mẫu)
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# =========================
# 4) "VECTOR HOÁ"/TIỀN XỬ LÝ (bước 3 của bài mẫu):
#    - Impute thiếu
#    - One-Hot cho Gender (categorical)
#    - Ghép lại bằng ColumnTransformer
# =========================
num_cols = ["Age", "EstimatedSalary"]
cat_cols = ["Gender"]

# OHE trả về array dày (dense) để vừa với GaussianNB
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn >= 1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn < 1.2

numeric_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])
categorical_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_cols),
        ("cat", categorical_tf, cat_cols),
    ]
)

# =========================
# 5) XÂY DỰNG & HUẤN LUYỆN MÔ HÌNH NAIVE BAYES (bước 4 của bài mẫu)
#    - GaussianNB phù hợp feature liên tục (Age/Salary)
# =========================
nb_clf = GaussianNB()

nb_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("nb", nb_clf)
])

nb_pipe.fit(X_train, y_train)

# =========================
# 6) ĐÁNH GIÁ HIỆU QUẢ (bước 5 của bài mẫu)
# =========================
y_pred = nb_pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
rep = classification_report(y_test, y_pred, digits=4)

print(f"Accuracy: {acc:.4f}")
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", rep)


    User ID  Gender  Age  EstimatedSalary  Purchased
0  15624510    Male   19            19000          0
1  15810944    Male   35            20000          0
2  15668575  Female   26            43000          0
3  15603246  Female   27            57000          0
4  15804002    Male   19            76000          0

Thiếu dữ liệu theo cột:
 User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64
Accuracy: 0.8750
Confusion Matrix:
 [[47  4]
 [ 6 23]]
Classification Report:
               precision    recall  f1-score   support

           0     0.8868    0.9216    0.9038        51
           1     0.8519    0.7931    0.8214        29

    accuracy                         0.8750        80
   macro avg     0.8693    0.8573    0.8626        80
weighted avg     0.8741    0.8750    0.8740        80

